In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.train import *
from fgc_support_retri.utils import *

I0405 13:13:16.141911 139978298066752 file_utils.py:39] PyTorch version 1.1.0 available.


In [3]:
def get_target_s(sp_list, s_len, window=2, bidirectional=False):
    
    def is_single(sp_i, sp_list):
        if ((sp_i-1) in sp_list and (sp_i+1) in sp_list):
            return False
        else:
            return True
    
    out = []
    
    for sp_i in sp_list:
        if is_single(sp_i, sp_list):
            if bidirectional:
                for t_i in range(max(sp_i-window, 0), min(sp_i+window+1, s_len)):
                    if t_i not in sp_list:
                        out.append((sp_i, t_i))
            else:
                for t_i in range(max(sp_i-window, 0), sp_i):
                    if t_i not in sp_list:
                        out.append((sp_i, t_i))
    return out

In [4]:
get_target_s([1,2,5,6,10], 1000000000000000)

[(1, 0), (2, 0), (5, 3), (5, 4), (6, 4), (10, 8), (10, 9)]

In [26]:
data = json_load('prediction/ematch+sf_test.json')

In [27]:
data[18]['QUESTIONS'][0]

{'QID': 'D033Q02',
 'QTYPE': '進階題',
 'ATYPE': 'Object',
 'AMODE': ['Single-Span-Extraction'],
 'QTEXT': '「北美自由貿易協議2.0」的英文簡稱為何?',
 'QTEXT_CN': '「北美自由贸易协议2.0」的英文简称为何?',
 'SENTS': [{'text': '「北美自由贸易协议2.0」的英文简称为何?',
   'start': 0,
   'end': 21,
   'IE': {'NER': [{'id': 'D0-S0-M0',
      'string': '北美',
      'type': 'LOC',
      'char_b': 1,
      'char_e': 3},
     {'id': 'D0-S0-M1',
      'string': '2.0',
      'type': 'NUMBER',
      'char_b': 9,
      'char_e': 12},
     {'id': 'D0-S0-M2',
      'string': '英文',
      'type': 'DEMONYM',
      'char_b': 14,
      'char_e': 16}],
    'COREF': {},
    'RELATION': [],
    'TOKEN': [{'word': '「', 'char_b': 0, 'char_e': 1, 'pos': 'PU'},
     {'word': '北美', 'char_b': 1, 'char_e': 3, 'pos': 'NR'},
     {'word': '自由', 'char_b': 3, 'char_e': 5, 'pos': 'JJ'},
     {'word': '贸易', 'char_b': 5, 'char_e': 7, 'pos': 'NN'},
     {'word': '协议', 'char_b': 7, 'char_e': 9, 'pos': 'NN'},
     {'word': '2.0', 'char_b': 9, 'char_e': 12, 'pos': 'CD'},
     {'wo

In [28]:
len(data)

193

In [53]:
miss_t_proportion = 0 
hit_t_proportion = 0
miss_s_proportion = 0
all_hit_proportion = 0
d_len = 0

window = 3
bidirectional = True

for d in data:
    for q in d['QUESTIONS']:
        shint_len = len(q['SHINT']) 
        if shint_len == 0:
            q['miss_t'] = set()
            continue
        
        hit_s = 0
        miss_s = 0
        miss_t = 0
        hit_s_t = 0
        
        hit_s_list = []
        for sp_i in q['sp']:
            if sp_i in q['SHINT']:
                hit_s += 1
                hit_s_list.append(sp_i)
            else:
                miss_s += 1
                
        targets = get_target_s(q['sp'], len(d['SENTS']), window=window, bidirectional=bidirectional)
        targets_hit_s_t = get_target_s(hit_s_list, len(d['SENTS']), window=window, bidirectional=bidirectional)
        
        hit_t_set = set()
        for t in targets:
            if t[1] in q['SHINT']:
                hit_t_set.add(t[1])
                
        hit_s_t_set = set()
        for t in targets_hit_s_t:
            if t[1] in q['SHINT']:
                hit_s_t_set.add(t[1])
                
                
        target_set = set([t[1] for t in targets])

        miss_t_set = set(q['SHINT']) - set(q['sp']) - target_set
        miss_t = len(miss_t_set)
        q['miss_t'] = miss_t_set
        
        require_sp_len = len(set(q['SHINT']) - set(q['sp']) )
        if require_sp_len == 0:
            continue
            
        d_len += 1
        
        miss_t_proportion += miss_t / require_sp_len
        hit_t_proportion += len(hit_t_set) / require_sp_len
        miss_s_proportion += miss_s / shint_len
        all_hit_proportion += len(hit_s_t_set) / require_sp_len

In [54]:
d_len

104

In [55]:
miss_t_proportion / d_len

0.24342948717948715

In [56]:
hit_t_proportion / d_len

0.7565705128205127

In [57]:
miss_s_proportion / d_len

0.5976877289377289

In [58]:
all_hit_proportion / d_len

0.6533653846153845

In [52]:
for d in tqdm(data):
    for q in d['QUESTIONS']:
        if q['miss_t']:
            print(q['QID'])

100%|██████████| 193/193 [00:00<00:00, 91026.73it/s]

D003Q06
D033Q10
D033Q13
D039Q04
D039Q05
D039Q06
D039Q08
D069Q03
D069Q06
D087Q01
D087Q02
D093Q04
D093Q07
D105Q01
D105Q02
D105Q10
D117Q01
D183Q01
D201Q02
D201Q04
D201Q05
D249Q02
D249Q03
D249Q04
D255Q05
D255Q06
D255Q08
D255Q10
D261Q06
D267Q03
D285Q05
D303Q01
D309Q04
D309Q05
D309Q06
D315Q01
D315Q02
D315Q07
D315Q08


In [41]:
data[1]['QUESTIONS'][0]['miss_t']

set()